In [1]:
import numpy as np
import cv2 
from glob import glob
import matplotlib.pyplot as plt
from keras.applications import ResNet50
from tensorflow import keras
from keras.models import Model
import time
from keras_preprocessing.sequence import pad_sequences
from math import log, exp

In [9]:
0.6*0.3

0.18

In [17]:
log(0.3)+log(0.6)+log(0.3)

-2.918771232417863

In [18]:
log(0.3)+log(0.6)

-1.7147984280919268

In [3]:
img_feature_vectors = np.load('img_feature_vectors.npy', allow_pickle='TRUE').item()
captions_dict_encoded = np.load('captions_dict_encoded.npy', allow_pickle='TRUE').item()
vocab = np.load('vocab.npy', allow_pickle='TRUE').item()
# filenames_testing = np.load("dev/filenames_dev.npy", allow_pickle=True).item()

In [4]:
model = keras.models.load_model('models/image_caption_gen_epoch_6 (2).h5')
max_len = 40
vocab_inv = {v: k for k, v in vocab.items()}
images_path = 'Flickr_Data/Images/'
images = glob(images_path+'*.jpg')

resnet_model = ResNet50(include_top=True)
resnet_model = Model(inputs=resnet_model.input, outputs=resnet_model.layers[-2].output)

In [5]:
images[8003]

'Flickr_Data/Images\\892340814_bdd61e10a4.jpg'

In [ ]:
def generate_caption(img_feature_vector):
    caption = [1]
    next_word = None
    img_feature_vector = np.array([img_feature_vector])
    
    while next_word != vocab['eos'] and len(caption) != 40:
        output = model.predict([img_feature_vector, pad(caption)], verbose=0)
        next_word = np.argsort(output)[0][-1]
        caption.append(next_word)
        
    return caption

In [23]:
def generate_img_feature_vector(filepath):
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = img.reshape(1, 224, 224, 3)
    
    feature_vector = resnet_model.predict(img, verbose=0).reshape(1, 2048)
    
    return feature_vector

In [24]:
def decode_caption(encoded_caption):
    decoded_caption = []
    for word in encoded_caption:
        if word == 0:
            continue
        decoded_caption.append(vocab_inv[word])
    return " ".join(decoded_caption)

In [ ]:
def image_caption_generator(filename, model):
    return decode_caption(generate_caption(img_feature_vectors[filename]))

In [ ]:
def show_image(filepath):
    plt.figure()
    img = cv2.imread(filepath)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

In [ ]:
images[8000].split("\\")[-1]

In [ ]:
start_time = time.time()

print(image_caption_generator(images[8000].split("\\")[-1], model))

time.time()-start_time

In [ ]:
show_image('Flickr_Data/Images\\'+filename1)

30 images results

In [ ]:
    results = np.load("results/results_8k_new_show_and_tell.npy", allow_pickle=True).item()

In [28]:
from IPython.display import display, Image
# for item in results:
#     print(results[item])
#     display(Image(filename='file/kaggle/working/'+item))

In [ ]:
from IPython.display import display, Image
for item in results:
    print(item)
    display(Image(filename='file/kaggle/working/'+results[item]))

In [ ]:
fig = plt.figure(figsize=(150,150))
  
# setting values to rows and column variables
rows = 30
columns = 1

i=1
for x,y in results.items():
    fig.add_subplot(rows, columns, i)
    plt.imshow(y)
    plt.axis('off')
    plt.title(x)
    i+=1

Beam search

In [21]:
def pad(alist):
    return np.array([pad_sequences([alist], maxlen=40, truncating='post')[0]])

In [ ]:
k=3
candidates = []
current_nodes = [([1], 1)]
current_nodes_copy = current_nodes[:]

while current_nodes != []:
    current_nodes = []
    for x, y in current_nodes_copy:
#         print(current_nodes_copy)
#         print()
        output = model.predict([img_feature_vector, pad(x)], verbose=0)
        next_words = np.argsort(output)[0][-k:]
        for item in next_words:
            new_prob = log(output[0][item])+y
            if item == vocab['eos'] or len(x)==40:
                candidates.append((x+[item],new_prob))
            else:
                current_nodes.append((x+[item],new_prob))
#             print(x)
#             print()
#             print((x+[item],new_prob))
#             print()
    current_nodes_copy = sorted(current_nodes, key=lambda tup:tup[1])[-k:]

In [ ]:
a = sorted(candidates, key=lambda tup:tup[1])[-10:]

In [ ]:
# smaller captions will have a higher combined probability

In [ ]:
for item in a:
    print(decode_caption(item[0]), item[1])

Beam Search 2

In [6]:
img_feature_vector = generate_img_feature_vector(images[8003])
fv = generate_img_feature_vector(images[8003])
display(Image(filename=images[8003]))

NameError: name 'generate_img_feature_vector' is not defined

In [96]:
def pad(alist):
    return np.array([pad_sequences([alist], maxlen=40, truncating='post')[0]])

In [167]:
def beam_search(fv, model, beam_size, vocab):
    complete_captions = []
    captions_tree = [
        [([vocab['sos']], 1)]
    ]
    for i in range(40):
        caps_to_be_expanded = captions_tree[i]
        for item in caps_to_be_expanded:
            if item[0][-1] == vocab['eos']:
                complete_captions.append(item)
        caps_to_be_expanded = sorted(filter(lambda t: t[0][-1] != vocab['eos'], caps_to_be_expanded), key=lambda t: t[1])
        caps_to_be_expanded = caps_to_be_expanded[-beam_size:]
        candidates = []
        if len(caps_to_be_expanded) == 0:
            return
        for caption, prob in caps_to_be_expanded:
            output = model.predict([fv, pad(caption)], verbose=0)
            # output = 2d array
            next_words = np.argsort(output)[0][-beam_size:]
            for word in next_words:
                new_caption = caption + [word]
                new_prob = (log(output[0][word])+prob)*(1/len(new_caption)**1)
                candidates.append((new_caption, new_prob))
        captions_tree.append(candidates)
    return complete_captions

In [168]:
a = beam_search(fv, model, 3, vocab)

In [169]:
# flat_list = [item for sublist in a for item in sublist]

# Sort the flattened list by the second element of each tuple
sorted_list = sorted(a, key=lambda x: x[1])

In [170]:
for item in sorted_list[-20:]:
    print(decode_caption(item[0]))
    print(final_cap[1])
    print()

sos a man in a red shirt is climbing a sheer cliff face with a large tree in his mouth . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in the background . . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in his hand . . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in his mouth . . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in the background . . . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in his hand . . . . . . . . . . . . . . . . . . eos
-6.580594631794981e-05

sos a man in a red shirt is climbing a sheer cliff face with a large tree in his mouth . . . . .

BLEU

In [ ]:
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu
import numpy as np

In [ ]:
captions_dict = np.load("30k/captions_dict.npy", allow_pickle=True).item()
model = keras.models.load_model('models/image_caption_gen_v1.h5')
filenames_test = np.load("30k/filenames_testing.npy", allow_pickle=True).item()

In [ ]:
def compute_bleu(filepath, captions_dict, model):
    ref = []
    for item in captions_dict[filepath.split("\\")[1]]:
        ref.append(item.split()[1:-1])
    cand = decode_caption(generate_caption(generate_img_feature_vector(filepath))).split()[1:-1]
    return np.array((sentence_bleu(ref, cand, weights=(1.0, 0, 0, 0)), 
    sentence_bleu(ref, cand, weights=(0.5, 0.5, 0, 0)),
    sentence_bleu(ref, cand, weights=(0.3, 0.3, 0.3, 0)),
    sentence_bleu(ref, cand, weights=(0.25, 0.25, 0.25, 0.25))))

In [ ]:
scores = []
n=0
filenames_test = list(filenames_test)[:500]
for item in filenames_test:
    filepath = "Flickr_Data/Images\\"+item
    result = compute_bleu(filepath, captions_dict, model)
    scores.append(result)
    print(n)
    n+=1

In [ ]:
avg_bleu = np.array(scores).mean(axis=0)

In [ ]:
avg_bleu

Top-k Sampling and Nucleus Sampling

In [ ]:
from random import sample
import random
from sklearn.preprocessing import normalize

In [ ]:
def apply_temp(softmax, temperature):
    softmax = np.log(softmax) / temperature
    softmax = np.exp(softmax)
    softmax = softmax / np.sum(softmax)
    return softmax

In [ ]:
def top_k(softmax, k, temp):
    x = np.argsort(softmax)
    for item in x[:-k]:
        softmax[item] = 0
    softmax = normalize([softmax], norm='l1')[0]
    softmax = apply_temp(softmax, temp)
    next_word = random.choices(list(range(0, len(softmax))), weights = softmax, k=1)[0]
    
    return next_word

In [ ]:
def generate_caption_top_k(img_feature_vector):
    caption = [1]
    next_word = None
    
    while next_word != vocab['eos'] and len(caption) != 40:
        output = model.predict([img_feature_vector, pad(caption)], verbose=0)
        
        next_word = top_k(output[0], 10, 0.5)
        
        caption.append(next_word)
        
    return caption

In [ ]:
def nucleus_sampling(softmax, p, temp):
    actual_p = 0
    x = np.argsort(softmax)
    for item in x[::-1]:
        if actual_p < p:
            actual_p = actual_p+softmax[item]
        else:
            softmax[item] = 0
    softmax = normalize([softmax], norm='l1')[0]
    softmax = apply_temp(softmax, temp)
    next_word = random.choices(list(range(0, len(softmax))), weights = softmax, k=1)[0]
    
    return next_word

In [ ]:
def generate_caption_nucleus_sampling(img_feature_vector):
    caption = [1]
    next_word = None

    while next_word != vocab['eos'] and len(caption) != 40:
        output = model.predict([img_feature_vector, pad(caption)], verbose=0)

        next_word = nucleus_sampling(output[0], 0.9, 0.5)

        caption.append(next_word)

    return caption

In [ ]:
def image_caption_generator_top_k(filepath, model):
    if type(filepath) == int:
        filepath = images[filepath]
    return decode_caption(generate_caption_top_k(generate_img_feature_vector(filepath)))

In [ ]:
def image_caption_generator_nucleus_sampling(filepath, model):
    if type(filepath) == int:
        filepath = images[filepath]
    return decode_caption(generate_caption_nucleus_sampling(generate_img_feature_vector(filepath)))

In [ ]:
show_image(8009)

In [ ]:
image_caption_generator_nucleus_sampling(8009, model)

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']], [['hello', 'there'], ['hi', 'there']]]
candidates = [['this', 'is', 'a', 'test'], ['hello', 'there']]
score = corpus_bleu(references, candidates)
print(score)

In [5]:
import string

In [14]:
x = '''the dog 's are , running !across the dark . # " hello " sky. a dog & a dog'''
x = x.split()
y = x[:]
for i, item in enumerate(y):
    if item in string.punctuation:
        if item == '&':
            x[i] = "and"
        elif item == '"':
            pass
        elif item == "#":
            pass
        else:
            x[i] = None 

z = []
for item in x:
    if item != None:
        z.append(item)
" ".join(z)

'the dog \'s are running !across the dark # " hello " sky. a dog and a dog'